In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='6'
import time
import numpy as np
import tensorflow as tf
from VGG16_GAP import VGG16_GAP
from VGG16_flatten import VGG16_flatten

In [ ]:
import numpy as np
import pandas as pd
import skimage.io as imageio
import pickle

In [ ]:
from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display

In [ ]:
with open('save/label_dict.pkl', 'rb') as f:
    y_dict = pickle.load(f)

In [ ]:
with open('save/inv_label_dict.pkl', 'rb') as f:
    inv_y_dict = pickle.load(f)

In [ ]:
HOME_DIR = "/home/cmchang/DLCV2018SPRING/final/"
TRAIN_DIR = HOME_DIR+"dlcv_final_2_dataset/train/"
VALID_DIR = HOME_DIR+"dlcv_final_2_dataset/val/"

In [ ]:
dtrain = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/train_id.txt", header=None,sep=" ", names=["img", "id"])
dvalid = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/val_id.txt", header=None,sep=" ", names=["img", "id"])

In [ ]:
train_list = list(TRAIN_DIR+dtrain.img)
valid_list = list(VALID_DIR+dvalid.img)

In [ ]:
def readImgList(file_list):
    images = list()
    for i, file in enumerate(file_list):
        print(i, end="\r")
        img = imageio.imread(file)
        img = img.astype(int)
        images.append(img)
    return np.array(images)

In [ ]:
def transformLabel(id_list, y_dict):
    label = list()
    for uid in list(id_list):
        label.append(y_dict[uid])
    return np.array(label)

In [ ]:
def one_hot_encoding(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

In [ ]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [ ]:
Xtrain = readImgList(train_list)
print("train:", Xtrain.shape)

In [ ]:
Xvalid = readImgList(valid_list)
print("valid:", Xtrain.shape)

In [ ]:
ytrain = transformLabel(list(dtrain.id), y_dict)

In [ ]:
yvalid = transformLabel(list(dvalid.id), y_dict)

In [ ]:
Ytrain = one_hot_encoding(ytrain, len(y_dict))
Yvalid = one_hot_encoding(yvalid, len(y_dict))

In [ ]:
scope_name = "Model"

In [ ]:
model = VGG16_GAP(scope_name=scope_name)

In [ ]:
FLAG_save_dir = "/home/cmchang/DLCV2018SPRING/final/newCL_lambda-1e-1_dynamic_gap_L5_v3_rescale0-1_save_linear/"
FLAG_init_from = FLAG_save_dir + "para_dict.npy"

In [ ]:
model.build(vgg16_npy_path=FLAG_init_from,
            shape=Xtrain.shape[1:],
            classes=len(y_dict),
            conv_pre_training=True,
            fc_pre_training=True,
            new_bn=False)

In [ ]:
dp = [1.0]
model.set_idp_operation(dp=dp)

In [ ]:
def count_number_params(para_dict):
    n = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'conv' in k or 'fc' in k:
            n += get_params_shape(v[0].shape.as_list())
            n += get_params_shape(v[1].shape.as_list())
    return n

def get_params_shape(shape):
    n = 1
    for dim in shape:
        n = n*dim
    return n

def count_flops(para_dict, net_shape):
    input_shape = (3 ,32 ,32) # Format:(channels, rows,cols)
    total_flops_per_layer = 0
    input_count = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'fc' in k:
            continue
        elif 'conv' in k:
            conv_filter = v[0].shape.as_list()[3::-1] # (64 ,3 ,3 ,3)  # Format: (num_filters, channels, rows, cols)
            stride = 1
            padding = 1

            if conv_filter[1] == 0:
                n = conv_filter[2] * conv_filter[3] # vector_length
            else:
                n = conv_filter[1] * conv_filter[2] * conv_filter[3]  # vector_length

            flops_per_instance = n + ( n -1)    # general defination for number of flops (n: multiplications and n-1: additions)

            num_instances_per_filter = (( input_shape[1] - conv_filter[2] + 2 * padding) / stride) + 1  # for rows
            num_instances_per_filter *= ((input_shape[1] - conv_filter[2] + 2 * padding) / stride) + 1  # multiplying with cols

            flops_per_filter = num_instances_per_filter * flops_per_instance
            total_flops_per_layer += flops_per_filter * conv_filter[0]  # multiply with number of filters

            total_flops_per_layer += conv_filter[0] * input_shape[1] * input_shape[2]

            input_shape = net_shape[input_count].as_list()[3:0:-1]
            input_count +=1

    total_flops_per_layer += net_shape[-1].as_list()[1] *2360*2
    return total_flops_per_layer

def countFlopsParas(net):
    total_flops = count_flops(net.para_dict, net.net_shape)
    if total_flops / 1e9 > 1:   # for Giga Flops
        print(total_flops/ 1e9 ,'{}'.format('GFlops'))
    else:
        print(total_flops / 1e6 ,'{}'.format('MFlops'))

    total_params = count_number_params(net.para_dict)

    if total_params / 1e9 > 1:   # for Giga Flops
        print(total_params/ 1e9 ,'{}'.format('G'))
    else:
        print(total_params / 1e6 ,'{}'.format('M'))
    
    return total_flops, total_params

In [ ]:
flops, params = countFlopsParas(model)
FLAG_flops_M = flops/1e6
FLAG_params_M = params/1e6
print("Flops: %3f M, Paras: %3f M" % (flops/1e6, params/1e6))

In [ ]:
# extract features
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.global_variables())
    print("Initialized")
    output = []
    for i in range(int(Xtrain.shape[0]/200+1)):
        print(i, end="\r")
        st = i*200
        ed = min((i+1)*200, Xtrain.shape[0])
        prob = sess.run(model.features, feed_dict={model.x: Xtrain[st:ed,:], 
                                                    model.is_train: False})
        output.append(prob)

    for i in range(int(Xvalid.shape[0]/200+1)):
        print(i, end="\r")
        st = i*200
        ed = min((i+1)*200, Xvalid.shape[0])
        prob = sess.run(model.features, feed_dict={model.x: Xvalid[st:ed,:], 
                                                    model.is_train: False})
        output.append(prob)

In [ ]:
EX = np.concatenate(output,)
print(EX.shape)

In [ ]:
Y = np.concatenate([ytrain, yvalid])
print(Y.shape)

In [ ]:
centers = np.zeros((len(y_dict), EX.shape[1]))
for i in range(len(y_dict)):
    centers[i,:] = np.mean(EX[Y==i,:], axis=0)

In [ ]:
centers.shape

In [ ]:
np.save(arr=centers,file=FLAG_save_dir+"centers.npy")